In [1]:
%matplotlib inline
%reload_ext autoreload
%autoreload 2

import os
import re
import io
import itertools
import pprint

from bokeh.io import output_notebook, show
from bokeh.layouts import gridplot
from bokeh.plotting import figure
from bokeh.models import Range1d, Legend
from bokeh.palettes import all_palettes

import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import xarray as xr
import pymongo

import climatools.lblnew.bestfit_params as bestfits
from climatools.lblnew import setup_bestfit, setup_overlap
import climatools.lblnew.pipeline as pipe_lblnew
import climatools.cliradlw.setup as setup_cliradlw
import climatools.cliradlw.pipeline as pipe_cliradlw
from climatools.cliradlw.utils import mapband_new2old

import climatools.html.html as climahtml
from climatools.lblnew.dataio import *
from climatools.plot.plot import *

import IPython.display as display

In [2]:
output_notebook()

Loading BokehJS ...

In [3]:
def make_query(param=None):
    return {'param.' + name: value for name, value in param.items()}   

In [4]:
def show_html(s): display.display(display.HTML(s))
    
def show_markdown(s): display.display(display.Markdown(s))

In [5]:
def load_output_file(path_csv):
    '''
    Load lblnew output .csv file to xarray.Dataset
    
    Parameters
    ----------
    path_csv: str
              Path to the .csv file to be loaded.
    ds: xarray.Dataset
        Data in the input file in the form of an xarray.Dataset.
    '''
    toindex = ['i', 'band', 'pressure', 'igg', 'g']    
    df = pd.read_csv(path_csv, sep=r'\s+')
    df = df.set_index([i for i in toindex if i in df.columns])
    df = df.rename(columns={'sfu': 'flug',
                            'sfd': 'fldg',
                            'fnet': 'fnetg',
                            'coolr': 'coolrg'})
    ds = xr.Dataset.from_dataframe(df)

    for l in ('level', 'layer'):
        if l in ds.data_vars:
            if len(ds[l].dims) > 1:
                surface = {d: 0 for d in ds.dims if d != 'pressure'}
                coord_level = ds[l][surface]
                ds.coords[l] = ('pressure', coord_level)
            else:
                ds.coords[l] = ('pressure', ds[l])
    
    return ds

In [6]:
def fmt_cool(ds_in):
    ds = ds_in.copy(deep=True)
    dims = ['igg', 'g', 'i']
    for dim in dims:
        if dim in ds:
            if ds[dim].shape == (1,): ds = ds.squeeze(dim).drop(dim)
            elif ds[dim].shape == (): ds = ds.drop(dim)
    if 'band' in ds.dims:
        try:
            ds = ds.squeeze('band')
        except ValueError:
            ds = ds.sum('band')     
    return ds['coolrg']

def pltdata_cooling_1():
    colors = all_palettes['Set1'][4]
    data = [{'label': 'CLIRAD `coolr_bands`',
             'srs': fmt_cool(DS_COOL.sel(i=1)),
             'line_dash': 'dashed', 'line_width': 5,
             'color': colors[0], 'alpha': .6},
            {'label': 'CRD',
             'srs': fmt_cool(DS_COOL_CRD.sum('g')),
             'line_dash': 'solid', 'line_width': 1.5,
             'marker': 'circle', 'marker_size': 5,
             'color': colors[2], 'alpha': 1,},
            {'label': 'WGT igg=10 wgt_flux=2',
             'srs': fmt_cool(DS_COOL_WGT.sel(igg=10).sum('g')),
             'line_dash': 'solid', 'line_width': 3,
             'marker': 'square', 'marker_size': 5,
             'color': colors[3], 'alpha': .6}]
    return data

def nice_xlims(pltdata=None, prange=None):
    def get_slice(srs):
        return srs.sel(pressure=slice(*prange))
    srss = [d['srs'] for d in pltdata]
    vmin = min([get_slice(srs).min() for srs in srss])
    vmax = max([get_slice(srs).max() for srs in srss])
    dv = (vmax - vmin) * .01
    return float(vmin - dv), float(vmax + dv)

def plt_cool(pltdata=None):
    plin = plt_vert_profile_bokeh(pltdata=pltdata,
                                  y_axis_type='linear',
                                  prange=(50, 1050))
    plin.plot_width, plin.plot_height = 400, 500
    plog = plt_vert_profile_bokeh(pltdata=pltdata,
                                  y_axis_type='log',
                                  prange=(.01, 200))
    plog.plot_width, plog.plot_height = 400, 500
    everything = gridplot(plin, plog, ncols=2)
    return everything
    
def script_plt_cooling():
    pltdata = pltdata_cooling_1()
    p = plt_cool(pltdata=pltdata)
    show(p)
    show_markdown('''*FIGURE.* Cooling rate profiles.''')

In [7]:
def fmt_flux(ds_in, ilevels=None):
    ds = ds_in.copy(deep=True)
    
    for l in ('level', 'layer'):
        if l in ds.data_vars:
            if len(ds[l].dims) > 1:
                surface = {d: 0 for d in ds.dims if d != 'pressure'}
                coord_level = ds[l][surface]
                ds.coords[l] = ('pressure', coord_level)
            else:
                ds.coords[l] = ('pressure', ds[l])
    
    if ilevels: ds = ds.isel(pressure=ilevels)

    dims = ['i', 'igg', 'g']
    for dim in dims:
        if dim in ds:
            if ds[dim].shape == (1,): ds = ds.squeeze(dim).drop(dim)
            elif ds[dim].shape == (): ds = ds.drop(dim)
    
    if 'band' in ds.dims:
        try:
            ds = ds.squeeze('band').drop('band')
        except ValueError:
            ds = ds.sum('band')
                
    df = ds.to_dataframe()
    df = df.set_index('level', append=True)               
    return df

def tbdata_flux():
    tropopauses = {'mls': 40, 'saw': 45, 'trp': 37}
    atmpro = PARAM['atmpro']
    ilevels = [0, tropopauses[atmpro], -1]
    
    benchmark = {'label': 'CRD',
                 'df': fmt_flux(DS_FLUX_CRD.sum('g'), ilevels=ilevels)}
    
    tbs = [{'label': 'WGT igg=10',
            'df': fmt_flux(DS_FLUX_WGT.sum('g').sel(igg=10), 
                           ilevels=ilevels)},
           {'label': 'CLIRAD',
            'df': fmt_flux(DS_FLUX.sel(i=1), ilevels=ilevels)}]
    
    data = {'benchmark': benchmark, 'others': tbs}
    return data

def tb_flux(tbdata=None):
    benchmark = tbdata['benchmark']
    others = tbdata['others']
    
    def show_tb(d=None):
        display.display(d['df'])
        show_markdown(f'''*TABLE.*  Fluxes. {d['label']}''')
        
    def show_tbdiff(d0=None, d=None):
        df = d['df']
        df.index = d0['df'].index
        display.display(df - d0['df'])
        show_markdown(
            f'''*TABLE.*  Fluxes.  ({d['label']}) - ({d0['label']})''')
        
    show_tb(benchmark)
    for d in others: show_tb(d)
    for d in others: show_tbdiff(d=d, d0=benchmark)
        
def script_tb_flux():
    data = tbdata_flux()
    tb_flux(tbdata=data)

In [8]:
def gasbands():
    bmap = mapband_new2old()
    gases = [gas for gas, _ in PARAM['molecule'].items()] 
    bands = [bmap[b] for b in PARAM['band']]
    return list(itertools.product(gases, bands))

def print_bestfit_params():
    df = pd.DataFrame()
    for gas, band in gasbands():
        param = bestfits.kdist_params(molecule=gas, band=band)
        srs = pd.Series(param)
        df[f'({gas}, {band})'] = srs
    df.fillna('-', inplace=True)
    pd.set_option('display.max_colwidth', -1)
    display.display(df.sort_index())
    pd.reset_option('display.max_colwidth')
        
def print_input_param():
    df_clirad = pd.Series(PARAM, name='clirad run').to_frame()
    df_lblnew = pd.Series(PARAM_LBLNEW, name='lblnew run compared against').to_frame()
    df = pd.merge(df_clirad, df_lblnew, how='outer', 
                  left_index=True, right_index=True)
    df.fillna('-', inplace=True)
    pd.set_option('display.max_colwidth', -1)
    display.display(df)
    pd.reset_option('display.max_colwidth')
        
def print_lblnew_param():
    pd.set_option('display.max_colwidth', -1)
    display.display(pd.Series(PARAM_LBLNEW).to_frame())
    pd.reset_option('display.max_colwidth')

In [9]:
'''
Cooling rate profile by layer
'''
def script_coolr_bylayer():
    '''
    Displays cooling rate of each layer
    '''
    keys = ['CRD', 'CLIRAD `coolr_bands`']
    dss = [DS_COOL_CRD.sum('g'), DS_COOL.sel(i=1)]
    das = [fmt_cool(ds) for ds in dss]
    
    dfs = [da.to_dataframe() for da in das]
    df = pd.concat(dfs, axis=1, keys=keys)
    
    fname = 'coolr_bylayer.csv'
    df.to_csv(fname)
    html_download = '''
    Download: <a href="./{fname}">{fname}</a>'''.format(fname=fname)
    
    with pd.option_context('display.max_rows', 999):
        display.display(df)
        
    display.display(display.HTML(html_download))

In [10]:
'''
Fluxes by layer
'''
def script_flux_bylayer():
    '''
    Display fluxes of each layer
    '''
    keys = ['CRD', 'CLIRAD']
    dss = [DS_FLUX_CRD.sum('g'), DS_FLUX.sel(i=1)]
    dfs = [fmt_flux(ds) for ds in dss]
    
    df = pd.concat(dfs, axis=1, keys=keys)
    
    fname = 'fluxes_bylayer.csv'
    df.to_csv(fname)
    html_download = '''
    Download: <a href="./{fname}">{fname}</a>'''.format(fname=fname)
    
    with pd.option_context('display.max_rows', 999):
        display.display(df)
        
    display.display(display.HTML(html_download))

In [11]:
def script_fluxcoolr_excelfile():
    '''
    Save fluxes and cooling rates by layer by g-interval in excel file
    and make it available for download.
    '''
    fname = 'output_byg.xlsx'
    
    writer = pd.ExcelWriter(fname)
    
    for g in DS_FLUX_WGT.coords['g']:
        df = DS_FLUX_WGT.sel(igg=1).sel(g=g).to_dataframe()
        df.to_excel(writer, f'flux g={int(g)}')
        
    for g in DS_COOL_WGT.coords['g']:
        df = DS_COOL_WGT.sel(igg=1).sel(g=g).to_dataframe()
        df.to_excel(writer, f'cool g={int(g)}')    

In [12]:


    
def script():
    
    title = '### CLIRAD-LW'
    s_input = 'Input Parameters'
    s_bestfits = 'Best-fit Parameters'
    s_plt_cooling = 'Cooling Rate Profiles'
    s_tb_flux = 'Flux Comparison'
    s_coolr_bylayer = 'Cooling Rate by Layer'
    s_flux_bylayer = 'Fluxes by Layer'
    
    # title
    show_markdown('### CLIRAD-LW')
    
    # TOC
    show_markdown('### Table of Contents')
    show_html(climahtml.getHTML_hrefanchor(s_input))
    show_html(climahtml.getHTML_hrefanchor(s_bestfits))
    show_html(climahtml.getHTML_hrefanchor(s_plt_cooling))
    show_html(climahtml.getHTML_hrefanchor(s_tb_flux))
    show_html(climahtml.getHTML_hrefanchor(s_coolr_bylayer))
    show_html(climahtml.getHTML_hrefanchor(s_flux_bylayer))
    
    # Input parameters
    show_html(climahtml.getHTML_idanchor(s_input))
    show_markdown(climahtml.getMarkdown_sectitle(s_input))
    print_input_param()
    
    # Best-fit values
    show_html(climahtml.getHTML_idanchor(s_bestfits))
    show_markdown(climahtml.getMarkdown_sectitle(s_bestfits))
    print_bestfit_params()
    
    # Cooling rate profiles
    show_html(climahtml.getHTML_idanchor(s_plt_cooling))
    show_markdown(climahtml.getMarkdown_sectitle(s_plt_cooling))
    script_plt_cooling()
    
    # Flux comparison
    show_html(climahtml.getHTML_idanchor(s_tb_flux))
    show_markdown(climahtml.getMarkdown_sectitle(s_tb_flux))
    script_tb_flux()
    
    # Cooling rate by layer
    show_html(climahtml.getHTML_idanchor(s_coolr_bylayer))
    show_markdown(climahtml.getMarkdown_sectitle(s_coolr_bylayer))
    script_coolr_bylayer()
    
    # Fluxes by layer
    show_html(climahtml.getHTML_idanchor(s_flux_bylayer))
    show_markdown(climahtml.getMarkdown_sectitle(s_flux_bylayer))
    script_flux_bylayer()

In [13]:
%run param.py

client = pymongo.MongoClient('localhost', 27017)

doc = client.lblnew.bestfit_lw.find_one(make_query(param=PARAM_LBLNEW))
DS_FLUX_CRD = load_output_file(io.StringIO(doc['output_fluxg']))
DS_COOL_CRD = load_output_file(io.StringIO(doc['output_coolrg']))
DS_FLUX_WGT = load_output_file(io.StringIO(doc['output_wfluxg']))
DS_COOL_WGT = load_output_file(io.StringIO(doc['output_wcoolrg']))

doc = client.cliradnew.lw.find_one(make_query(param=PARAM))
DS_FLUX = load_output_file(io.StringIO(doc['output_flux']))
DS_COOL = load_output_file(io.StringIO(doc['output_coolr']))

In [14]:
script()
if 'ng_refs' in PARAM_LBLNEW:
    script_fluxcoolr_excelfile()

### CLIRAD-LW

### Table of Contents

# Input Parameters

,clirad run,lblnew run compared against
atmpro,mls,mls
band,[9],7
commitnumber,1013f91,a06b618
conc,-,-
dv,-,0.001
klin,-,0
molecule,{'h2o': 'atmpro'},h2o
ng_adju,-,[0]
ng_refs,-,[7]
nv,-,1000


# Best-fit Parameters

...

*TABLE.*  Fluxes.  (CLIRAD) - (CRD)

# Cooling Rate by Layer

CRD           CLIRAD `coolr_bands`          
            layer    coolrg                layer    coolrg
pressure                                                  
0.000312        1 -0.003586                    1 -0.000078
0.000750        2 -0.003578                    2 -0.000084
0.001052        3 -0.003559                    3 -0.000030
0.001476        4 -0.003536                    4  0.000006
0.002070        5 -0.003511                    5  0.000034
0.002904        6 -0.003485                    6  0.000069
0.004074        7 -0.003457                    7  0.000100
0.005714        8 -0.003431                    8  0.000136
0.008015        9 -0.003403                    9  0.000171
0.011243       10 -0.003376                   10  0.000209
0.015771       11 -0.003350                   11  0.000248
0.022122       12 -0.003329                   12  0.000287
0.031031       13 -0.003315                   13  0.000329
0.043528       14 -0.003309                   14  0.000371
0.061057       15 -0.003206                   15  0.000512
0.085645       16 -0.002705                   16  0.000984
0.120136       17 -0.001532                   17  0.001904
0.168516       18  0.000364                   18  0.003250
0.236378       19  0.003208                   19  0.005145
0.331549       20  0.007301                   20  0.007772
0.465100       21  0.013032                   21  0.011374
0.652400       22  0.020888                   22  0.016281
0.915100       23  0.028761                   23  0.021255
1.283650       24  0.028229                   24  0.021716
1.800600       25  0.020295                   25  0.017792
2.525700       26  0.013084                   26  0.013578
3.542800       27  0.008076                   27  0.010081
4.969550       28  0.004895                   28  0.007365
6.970850       29  0.003042                   29  0.005355
9.778100       30  0.001969                   30  0.003857
13.715850      31  0.001262                   31  0.002678
19.239350      32  0.000763                   32  0.001757
26.987250      33  0.000492                   33  0.001131
37.855300      34  0.000360                   34  0.000719
53.100050      35  0.000204                   35  0.000371
73.887500      36  0.000017                   36  0.000059
97.662500      37 -0.000096                   37 -0.000130
121.437500     38 -0.000149                   38 -0.000227
145.212500     39 -0.000234                   39 -0.000321
168.987500     40 -0.000288                   40 -0.000371
192.762500     41 -0.000166                   41 -0.000299
216.537500     42  0.000813                   42  0.000477
240.312500     43  0.003293                   43  0.002607
264.087500     44  0.007068                   44  0.006265
287.862500     45  0.011686                   45  0.011800
311.637500     46  0.015158                   46  0.014854
335.412500     47  0.019409                   47  0.017892
359.187500     48  0.025189                   48  0.023308
382.962500     49  0.032281                   49  0.030902
406.737500     50  0.040512                   50  0.039329
430.512500     51  0.049589                   51  0.047627
454.287500     52  0.058272                   52  0.054743
478.062500     53  0.067152                   53  0.062713
501.837500     54  0.076627                   54  0.072297
525.612500     55  0.086571                   55  0.083638
549.387500     56  0.096936                   56  0.095833
573.162500     57  0.107898                   57  0.108121
596.937500     58  0.123729                   58  0.124200
620.712500     59  0.138122                   59  0.137540
644.487500     60  0.149943                   60  0.147415
668.262500     61  0.159523                   61  0.155251
692.037500     62  0.167059                   62  0.161925
715.812500     63  0.173049                   63  0.170035
739.587500     64  0.177992                   64  0.178699
763.362500     65  0.182396                   65  0.186801
78

# Fluxes by Layer

CRD                              CLIRAD  \
                        flug          fldg      fnetg       flug   
pressure    level                                                  
0.000000    1     -16.108744  0.000000e+00 -16.108744 -16.411338   
0.000624    2     -16.108744  8.405096e-08 -16.108744 -16.411338   
0.000876    3     -16.108744  1.184119e-07 -16.108744 -16.411338   
0.001229    4     -16.108744  1.681485e-07 -16.108744 -16.411338   
0.001723    5     -16.108744  2.402944e-07 -16.108744 -16.411338   
0.002417    6     -16.108745  3.449184e-07 -16.108745 -16.411338   
0.003391    7     -16.108745  4.966543e-07 -16.108745 -16.411338   
0.004757    8     -16.108746  7.166952e-07 -16.108745 -16.411338   
0.006672    9     -16.108747  1.035446e-06 -16.108746 -16.411339   
0.009359    10    -16.108749  1.496943e-06 -16.108747 -16.411339   
0.013128    11    -16.108751  2.164873e-06 -16.108749 -16.411339   
0.018415    12    -16.108754  3.130662e-06 -16.108751 -16.411339   
0.025830    13    -16.108758  4.524967e-06 -16.108754 -16.411340   
0.036232    14    -16.108765  6.535416e-06 -16.108758 -16.411341   
0.050823    15    -16.108773  9.435279e-06 -16.108764 -16.411342   
0.071291    16    -16.108785  1.390238e-05 -16.108771 -16.411343   
0.100000    17    -16.108803  2.202975e-05 -16.108781 -16.411344   
0.140271    18    -16.108826  3.835314e-05 -16.108788 -16.411344   
0.196760    19    -16.108856  7.087237e-05 -16.108786 -16.411340   
0.275997    20    -16.108890  1.345888e-04 -16.108755 -16.411326   
0.387100    21    -16.108917  2.571929e-04 -16.108659 -16.411291   
0.543100    22    -16.108907  4.888673e-04 -16.108419 -16.411212   
0.761700    23    -16.108793  9.156824e-04 -16.107878 -16.411040   
1.068500    24    -16.108462  1.629992e-03 -16.106832 -16.410713   
1.498800    25    -16.107925  2.531439e-03 -16.105394 -16.410240   
2.102400    26    -16.107328  3.385410e-03 -16.103942 -16.409709   
2.949000    27    -16.106734  4.104154e-03 -16.102630 -16.409160   
4.136600    28    -16.106169  4.674716e-03 -16.101494 -16.408616   
5.802500    29    -16.105645  5.116897e-03 -16.100528 -16.408096   
8.139200    30    -16.105164  5.477822e-03 -16.099686 -16.407614   
11.417000   31    -16.104729  5.807610e-03 -16.098922 -16.407192   
16.014700   32    -16.104375  6.140180e-03 -16.098234 -16.406877   
22.464000   33    -16.104164  6.512633e-03 -16.097652 -16.406734   
31.510500   34    -16.104143  7.018756e-03 -16.097124 -16.406815   
44.200100   35    -16.104360  7.776293e-03 -16.096584 -16.407193   
62.000000   36    -16.104997  8.842846e-03 -16.096155 -16.408058   
85.775000   37    -16.106333  1.022689e-02 -16.096106 -16.409654   
109.550000  38    -16.108049  1.167256e-02 -16.096376 -16.411580   
133.325000  39    -16.110045  1.324828e-02 -16.096797 -16.413743   
157.100000  40    -16.112353  1.489865e-02 -16.097454 -16.416144   
180.875000  41    -16.115052  1.678758e-02 -16.098264 -16.418839   
204.650000  42    -16.119238  2.050722e-02 -16.098730 -16.422903   
228.425000  43    -16.129234  3.279367e-02 -16.096441 -16.432635   
252.200000  44    -16.149114  6.194755e-02 -16.087166 -16.451975   
275.975000  45    -16.184206  1.169452e-01 -16.067260 -16.485904   
299.750000  46    -16.241225  2.068765e-01 -16.034349 -16.540789   
323.525000  47    -16.316120  3.244613e-01 -15.991658 -16.612750   
347.300000  48    -16.407203  4.702079e-01 -15.936996 -16.700029   
371.075000  49    -16.515533  6.494763e-01 -15.866056 -16.803770   
394.850000  50    -16.641891  8.667474e-01 -15.775144 -16.924687   
418.625000  51    -16.786742  1.125691e+00 -15.661051 -17.063188   
442.400000  52    -16.950511  1.429120e+00 -15.521391 -17.219700   
466.175000  53    -17.130402  1.773123e+00 -15.357279 -17.391768   
489.950000  54    -17.325032  2.156873e+00 -15.168159 -17.577673   
513.725000  55    -17.534363  2.582011e+00 -14.952353 -17.777500   
537.500000  56    -17.758188  3.049648e+00 -14.708540 -17.991294   
561.275000  5

In [15]:
display.HTML('''<script>
code_show=true; 
function code_toggle() {
 if (code_show){
 $('div.input').hide();
 } else {
 $('div.input').show();
 }
 code_show = !code_show
} 
$( document ).ready(code_toggle);
</script>
<form action="javascript:code_toggle()"><input type="submit" value="Click here to toggle on/off the raw code."></form>''')